In [16]:
from datasets import load_dataset
from LatentPixel import LatentLlama, TGraph, BIN_FONT
from tqdm import tqdm
import numpy as np

In [ ]:
'''
Xiyang's cirrus environment
'''
data = load_dataset('lambada', split='test')
_ = TGraph.init_render(
    dpi=80,
    font_file=BIN_FONT,
    font_size=8,
    pixels_per_patch=8,
    rgb=False,
    binary=True,
    max_seq_length=800,
    patch_len=2
)
model = LatentLlama(
    backbone_path='/work/sc118/sc118/shared/checkpoints/dllama_pretrain/lpixel_pretrain/LatentLlama/20231025-162146/last/backbone/',
    num_channels=1,
    patch_len=2,
    patch_size=8,
    binary=True
)

In [17]:
'''
Yintao's devbox environment
'''
data = load_dataset('lambada', split='test')
TGraph.init_render(
    dpi=80,
    font_file=BIN_FONT,
    font_size=8,
    pixels_per_patch=8,
    rgb=False,
    binary=True,
    max_seq_length=800,
    patch_len=2
)
model = LatentLlama(
    backbone_path='storage/llama_2_backbone/',
    num_channels=1,
    patch_len=2,
    patch_size=8,
    binary=True
)

[2023-11-24 01:08:46,587] [ WARNING] builder.py:816 - Found cached dataset lambada (/mnt/data/huggingface/lambada/plain_text/1.1.0/2e4879aaaa342d8f748b7275991006d2e27a8b0abc0a28ea299b3e3b839a3a40)


Coder path is none, do not load compressor for this model


Some weights of LlamaForPatchCausalInference were not initialized from the model checkpoint at storage/llama_2_backbone/ and are newly initialized: ['embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
def process_lambda(raw: str) -> str:
    raw = raw.replace('`` ', '"').replace(" ''", '"').replace('``', '"').replace("''", '"')
    result = ''
    for idx in range(len(raw) - 1):
        c = raw[idx]
        cn = raw[idx + 1]
        if cn in ',.?><{}()\'' and c == ' ':
            continue
        if (raw[idx+1:idx+4] == "n't" or raw[idx+1:idx+4] == "'re" or raw[idx+1:idx+3] == "'d" or raw[idx+1:idx+3] == "'s" or raw[idx+1:idx+3] == "'m") and raw[idx] == ' ':
            continue
        if raw[idx+1] == "'" and raw[idx+2] in 'abcdefghijklmnopqrstuvwxyz' and raw[idx] == ' ':
            continue
        result += raw[idx]
    result += raw[-1]
    return result

In [19]:
model.eval()
model.cuda()
generated = []
for txt in tqdm(data['text'][:200]):
    words = txt.split()
    prompt = process_lambda(' '.join(words[:-1]))
    prompt = TGraph.from_text([prompt])
    prompt = prompt._spacing_text(3)
    prompt.set_device('cuda')
    target = words[-1]
    gen = model.autoregressive_generate(prompt, None, 4)
    prompt.set_device('cpu')
    gen.set_device('cpu')
    generated.append((prompt, target, gen))

100%|██████████| 200/200 [00:08<00:00, 22.84it/s]


In [20]:
capital = '1 3 5 7 8 17 19 21 23 24 26 29 30 31 33 36 38 40 41 43 48 55 57 58 59 60 61 62 65 70 72 74 76 79 80 81 82 87 88 90 93 95 96 97 99 101 104 106 110 112 115 116 117 118 120 121 122 124 126 131 132 134 138 139 141 143 145 147 148 149 150 151 154 155 157 159 160 162 164 165 166 167 172 174 176 180 181 183 186 187 189 191 192 195 197 198 199'
wrong_word = '19 21 24 58 81 88 106 182 184'
correct = '2 4 20 35 63 111 141 145'
non_text = '0 1 3 5 6 7 8 9 10 11 12 13 14 15 16 17 18 22 23 25 26 27 28 29 30 31 32 33 34 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 59 60 61 62 64 65 66 67 68 69 70 72 71 73 74 75 76 77 78 79 80 82 83 84 85 86 87 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 107 108 109 110 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 142 143 144 146 147 148 149 150 151 152 153 154 155 156 157 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 183 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199'

In [21]:
capitals = set(capital.split(' '))
wrong_words = set(wrong_word.split(' '))
corrects = set(correct.split(' '))
non_texts = set(non_text.split(' '))

In [22]:
print('Acc: ', len(corrects) / 200)
print('Wrong: ', len(wrong_words) / 200)
print('Acc_no_cap: ', len(corrects - capitals) / (200 - len(capitals)))
print('Wrong_no_cap: ', len(wrong_words - capitals) / (200 - len(capitals)))
print('Non_text: ', len(non_texts) / 200)
print('Non_text_no_cap: ', len(non_texts - capitals) / (200 - len(capitals)))
print('Num_no_cap: ', (200 - len(capitals)))
print('Num_total: ', 200)

Acc:  0.04
Wrong:  0.045
Acc_no_cap:  0.05825242718446602
Wrong_no_cap:  0.019417475728155338
Non_text:  0.91
Non_text_no_cap:  0.912621359223301
Num_no_cap:  103
Num_total:  200


Add OCR

In [23]:
correct = '2 4 20 35 63 111 141 145 158'

In [25]:
idx = 20
pro = generated[idx][0]
t = generated[idx][1]
g   = generated[idx][2]
img: TGraph = g[pro._num_text_patches[0]-1:g._num_text_patches[0]-1]
text = img.ocr(method='paddleocr', square=False, span=True, span_ratio=1.3, scale=3)
print("Predictions:",text,"\nTarget:", t)

[2023/11/24 01:09:17] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.09052062034606934
[2023/11/24 01:09:18] ppocr DEBUG: rec_res num  : 1, elapsed : 0.08508086204528809
[[[[[30.0, 111.0], [217.0, 111.0], [217.0, 134.0], [30.0, 134.0]], ('track cna t', 0.6663442254066467)]]]
Predictions: ['track cna t'] 
Target: track


In [ ]:
img.to_PIL(square=False)[0]

In [ ]:
img.value.shape

In [ ]:
img._to_PIL(val[0])

In [ ]:
val.shape

In [ ]:
imgs = TGraph.from_text(['Portal']).to_PIL(square=False)
imgs = [np.array(im.resize((im.size[0]*3, im.size[1]*3))) for im in imgs]
ocr = PaddleOCR( det=False, cls = False, lang="en")

# results = [ocr.ocr(im, cls=True)[0][0][1][0].lower() for im in imgs]
results = [ocr.ocr(im, cls=True) for im in imgs]
results

In [ ]:
imgs.shape